# Day 14: Extended Polymerization

https://adventofcode.com/2021/day/14

In [1]:
example_txt = """NNCB

CH -> B
HH -> N
CB -> H
NH -> C
HB -> C
HC -> B
HN -> C
NN -> C
BH -> H
NC -> B
NB -> B
BN -> B
BB -> N
BC -> B
CC -> N
CN -> C"""

In [2]:
with open('input.txt') as input_file:
    input_txt = input_file.read()

In [3]:
def process_input(txt, lower=False):
    """Get the starting template and the pair insertion rules."""
    txt = txt.strip().split('\n\n')
    template = txt[0]
    rules = {}
    for line in txt[1].split('\n'):
        pair, insertion = line.split(' -> ')
        rules[pair] = pair[0] + (insertion.lower() if lower else insertion) + pair[1]
    return template, rules

## Part 1

In [4]:
def replace_step(template, rules):
    """Apply rules using simple replacements until there are no matches."""
    while any(pair in template for pair in rules.keys()):
        for pair in rules.keys():
            if pair in template:
                template = template.replace(pair, rules[pair])
    template = template.upper()  # convert insertion to upper case
    return template

In [5]:
%%time
for txt in (example_txt, input_txt):
    template, rules = process_input(txt, lower=True)
    for s in range(10):
        template = replace_step(template, rules)
        counts = [template.count(element) for element in set(template)]
        print(f'Step {s+1}: {max(counts) - min(counts)}.')
    print()

Step 1: 1.
Step 2: 5.
Step 3: 7.
Step 4: 18.
Step 5: 33.
Step 6: 82.
Step 7: 160.
Step 8: 366.
Step 9: 727.
Step 10: 1588.

Step 1: 8.
Step 2: 14.
Step 3: 23.
Step 4: 48.
Step 5: 99.
Step 6: 191.
Step 7: 389.
Step 8: 793.
Step 9: 1643.
Step 10: 3342.

CPU times: user 28.2 ms, sys: 4.2 ms, total: 32.4 ms
Wall time: 39.2 ms


The above solution doesn't scale for the 40 steps required in Part 2.  Try a modification using a regex.

In [6]:
import re
def regex_step(template, rules):
    """Apply rules using a regex until there are no matches."""
    pattern = re.compile('|'.join(rules.keys()))
    while any(pair in template for pair in rules.keys()):
        template = pattern.sub(lambda m : rules[m.group(0)], template)
    template = template.upper()  # convert insertion to upper case
    return template

In [7]:
%%time
for txt in (example_txt, input_txt):
    template, rules = process_input(txt, lower=True)
    for s in range(10):
        template = regex_step(template, rules)
        counts = [template.count(element) for element in set(template)]
        print(f'Step {s+1}: {max(counts) - min(counts)}.')
    print()

Step 1: 1.
Step 2: 5.
Step 3: 7.
Step 4: 18.
Step 5: 33.
Step 6: 82.
Step 7: 160.
Step 8: 366.
Step 9: 727.
Step 10: 1588.

Step 1: 8.
Step 2: 14.
Step 3: 23.
Step 4: 48.
Step 5: 99.
Step 6: 191.
Step 7: 389.
Step 8: 793.
Step 9: 1643.
Step 10: 3342.

CPU times: user 34.7 ms, sys: 5.15 ms, total: 39.8 ms
Wall time: 50.7 ms


This takes longer than the first simpler solution.  We need to consider a different approach for Part 2.

## Part 2

A more efficient solution can be found by only storing counts of each pair and each element instead of the full template.

In [8]:
def get_counts(template, rules):
    """Get the counts for each pair and each element in a template."""
    elements = set(''.join([key for key in rules.keys()]))
    pair_counts, element_counts = {}, {}
    for key in rules.keys():
        pair_counts[key] = template.count(key)
    for element in elements:
        element_counts[element] = template.count(element)
    return pair_counts, element_counts

In [9]:
def count_step(pair_counts, element_counts, rules):
    """Evaluate the new counts after a pair insertion step."""
    new_pair_counts = pair_counts.copy()
    for key, value in rules.items():  # comments below for a pair insertion rule AB -> C
        new_pair_counts[key] -= pair_counts[key]  # decrease the replaced pair (AB)
        new_pair_counts[value[:2]] += pair_counts[key]  # increase the new left pair (AC)
        new_pair_counts[value[1:]] += pair_counts[key]  # increase the new right pair (CB)
        element_counts[value[1]] += pair_counts[key]  # increase the middle element (C)
    return new_pair_counts, element_counts

In [10]:
%%time
for txt in (example_txt, input_txt):
    template, rules = process_input(txt)
    pair_counts, element_counts = get_counts(template, rules)
    for s in range(40):
        pair_counts, element_counts = count_step(pair_counts, element_counts, rules)
        counts = [template.count(element) for element in set(template)]
        print(f'Step {s+1}: {max(element_counts.values()) - min(element_counts.values())}.')
    print()

Step 1: 1.
Step 2: 5.
Step 3: 7.
Step 4: 18.
Step 5: 33.
Step 6: 82.
Step 7: 160.
Step 8: 366.
Step 9: 727.
Step 10: 1588.
Step 11: 3182.
Step 12: 6750.
Step 13: 13573.
Step 14: 28261.
Step 15: 56892.
Step 16: 117020.
Step 17: 235560.
Step 18: 480563.
Step 19: 966805.
Step 20: 1961318.
Step 21: 3942739.
Step 22: 7967209.
Step 23: 16003172.
Step 24: 32248617.
Step 25: 64726890.
Step 26: 130175202.
Step 27: 261104572.
Step 28: 524366212.
Step 29: 1051177797.
Step 30: 2108829309.
Step 31: 4225510980.
Step 32: 8470515455.
Step 33: 16966054976.
Step 34: 33990982425.
Step 35: 68061233152.
Step 36: 136300733814.
Step 37: 272851352872.
Step 38: 546243749493.
Step 39: 1093272152977.
Step 40: 2188189693529.

Step 1: 8.
Step 2: 14.
Step 3: 23.
Step 4: 48.
Step 5: 99.
Step 6: 191.
Step 7: 389.
Step 8: 793.
Step 9: 1643.
Step 10: 3342.
Step 11: 6820.
Step 12: 13818.
Step 13: 27915.
Step 14: 56273.
Step 15: 113290.
Step 16: 227539.
Step 17: 456575.
Step 18: 914812.
Step 19: 1831507.
Step 20: 3663959